### Ici nous allons proceder a l'optimisation des hyperparametres de nos 3 modeles d'apprentissage : XGBoost, LightGBM et CatBoost en utilisant la validation croisee d'ordre k et la tree structured parzen estimator 

In [2]:
# Importation des librairies nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import numpy as np

In [3]:
#creation d'un dataframe panda
df=pd.read_excel("Concrete_Data.xls")
df.head(20)

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075
5,266.0,114.0,0.0,228.0,0.0,932.0,670.0,90,47.029847
6,380.0,95.0,0.0,228.0,0.0,932.0,594.0,365,43.698299
7,380.0,95.0,0.0,228.0,0.0,932.0,594.0,28,36.447770
8,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.854291
9,475.0,0.0,0.0,228.0,0.0,932.0,594.0,28,39.289790


In [4]:
# Normalisation z-score ou standardisation
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()

normData=scaler.fit_transform(df)

normData

array([[ 2.47791782, -0.85688631, -0.84713204, ..., -1.21765845,
        -0.27973311,  2.64519215],
       [ 2.47791782, -0.85688631, -0.84713204, ..., -1.21765845,
        -0.27973311,  1.5612771 ],
       [ 0.49144303,  0.79553263, -0.84713204, ..., -2.24091255,
         3.55306569,  0.26660766],
       ...,
       [-1.2700575 ,  0.75958527,  0.85066095, ...,  0.08012724,
        -0.27973311, -0.7259282 ],
       [-1.16857975,  1.3080724 , -0.84713204, ...,  0.19118775,
        -0.27973311, -0.18264932],
       [-0.19401043,  0.30850389,  0.37696733, ..., -0.15072887,
        -0.27973311, -0.20461667]])

In [5]:
# Convertir le tableau obtenu en un dataframe
normData=pd.DataFrame(normData)
normData

,0,1,2,3,4,5,6,7,8
0,2.477918,-0.856886,-0.847132,-0.916663,-0.620225,0.863160,-1.217658,-0.279733,2.645192
1,2.477918,-0.856886,-0.847132,-0.916663,-0.620225,1.056170,-1.217658,-0.279733,1.561277
2,0.491443,0.795533,-0.847132,2.175367,-1.038944,-0.526514,-2.240913,3.553066,0.266608
3,0.491443,0.795533,-0.847132,2.175367,-1.038944,-0.526514,-2.240913,5.057677,0.313515
4,-0.790432,0.678414,-0.847132,0.488805,-1.038944,0.070531,0.647908,4.978487,0.507753
...,...,...,...,...,...,...,...,...,...
1025,-0.045623,0.488241,0.564569,-0.092122,0.451695,-1.323003,-0.065874,-0.279733,0.507051
1026,0.392837,-0.856886,0.960095,0.676201,0.702927,-1.994679,0.496916,-0.279733,-0.277827
1027,-1.270057,0.759585,0.850661,0.521599,-0.017270,-1.036061,0.080127,-0.279733,-0.725928
1028,-1.168580,1.308072,-0.847132,-0.279518,0.853666,0.214646,0.191188,-0.279733,-0.182649


In [6]:
req_col_names = ["Cement", "BlastFurnaceSlag", "FlyAsh", "Water", "Superplasticizer",
                 "CoarseAggregate", "FineAggregate", "Age", "CC_Strength"]
curr_col_names = list(normData.columns)

mapper = {}
for i, name in enumerate(curr_col_names):
    mapper[name] = req_col_names[i]

normData = normData.rename(columns=mapper)

normData.head()

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,CC_Strength
0,2.477918,-0.856886,-0.847132,-0.916663,-0.620225,0.863160,-1.217658,-0.279733,2.645192
1,2.477918,-0.856886,-0.847132,-0.916663,-0.620225,1.056170,-1.217658,-0.279733,1.561277
2,0.491443,0.795533,-0.847132,2.175367,-1.038944,-0.526514,-2.240913,3.553066,0.266608
3,0.491443,0.795533,-0.847132,2.175367,-1.038944,-0.526514,-2.240913,5.057677,0.313515
4,-0.790432,0.678414,-0.847132,0.488805,-1.038944,0.070531,0.647908,4.978487,0.507753


In [7]:
normData.describe()

,Cement,BlastFurnaceSlag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,CC_Strength
count,1.030000e+03,1030.000000,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03,1.030000e+03
mean,9.657862e-17,0.000000,2.759389e-17,3.587206e-16,-2.759389e-17,2.276496e-16,-1.931572e-16,2.069542e-17,2.759389e-16
std,1.000486e+00,1.000486,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00,1.000486e+00
min,-1.715219e+00,-0.856886,-8.471320e-01,-2.802333e+00,-1.038944e+00,-2.212137e+00,-2.240913e+00,-7.073594e-01,-2.005443e+00
25%,-8.500258e-01,-0.856886,-8.471320e-01,-7.808013e-01,-1.038944e+00,-5.265140e-01,-5.319534e-01,-6.123314e-01,-7.252985e-01
50%,-7.912996e-02,-0.601776,-8.471320e-01,1.608624e-01,2.460198e-02,-6.328926e-02,7.401267e-02,-2.797331e-01,-8.235105e-02
75%,6.589771e-01,0.800751,1.001836e+00,4.888051e-01,6.627297e-01,7.267662e-01,6.291904e-01,1.637312e-01,6.179613e-01
max,2.477918e+00,3.310688,2.281122e+00,3.065497e+00,4.354156e+00,2.214232e+00,2.733101e+00,5.057677e+00,2.801689e+00


In [8]:
X = normData.iloc[:,:-1]
y = normData.iloc[:,-1]

In [19]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
kf=KFold(n_splits=10,shuffle=True,random_state=42)

cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 927, Test set:103
Fold:2, Train set: 927, Test set:103
Fold:3, Train set: 927, Test set:103
Fold:4, Train set: 927, Test set:103
Fold:5, Train set: 927, Test set:103
Fold:6, Train set: 927, Test set:103
Fold:7, Train set: 927, Test set:103
Fold:8, Train set: 927, Test set:103
Fold:9, Train set: 927, Test set:103
Fold:10, Train set: 927, Test set:103


In [20]:
import xgboost as xgb
from xgboost import XGBRegressor

xgbr = XGBRegressor()

score = cross_val_score(xgbr, X, y, cv= kf, scoring="neg_mean_squared_error")

In [21]:
score

array([-0.06513576, -0.05764215, -0.06972757, -0.07478705, -0.10023822,
       -0.04120526, -0.04804072, -0.05500443, -0.04447825, -0.05458346])

In [76]:
def rmse(score):
    rmse = np.sqrt(-score)
    # print(f'rmse= {"{:.3f}".format(rmse)}')
    return rmse

In [23]:
#rmse(score.mean())

rmse= 0.247


In [24]:
import lightgbm as lgbm
from lightgbm import LGBMRegressor
lgb=LGBMRegressor()

score2 = cross_val_score(lgb,X,y,cv=kf,scoring="neg_mean_squared_error")
score2

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005224 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1066
[LightGBM] [Info] Number of data points in the train set: 927, number of used features: 8
[LightGBM] [Info] Start training from score 0.009573
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000088 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1026
[LightGBM] [Info] Number of data points in the train set: 927, number of used features: 8
[LightGBM] [Info] Start training from score -0.007450
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000114 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1074
[LightGBM] [Info] Number of data points in the train

array([-0.05911354, -0.08261411, -0.08267357, -0.0866818 , -0.11042436,
       -0.04744557, -0.05844121, -0.07067186, -0.04703379, -0.0462187 ])

In [25]:
rmse(score2.mean())

rmse= 0.263


#### XGBoost

In [77]:
# Defining hyperparameter search space for the three models
from hyperopt import hp

search_space_xgb={
    "alpha":hp.uniform("alpha",0.05,1), #L1 regularization term on weigths
    "lambda":hp.uniform("lambda",0.05,1), #L2 regularization term on weigths 
    "eta":hp.uniform("eta",0.01,0.3),# learning rate
    "max_depth":hp.randint("max_depth",2,30),
    "subsample":hp.uniform("subsample",0.5,1),
    "max_bin":hp.randint("max_bin",10,2000),# maximum number of descrete bins  to bucket continuous features
    "gamma":hp.uniform("gamma",0,1), # min spleat loss
    "min_child_weight":hp.uniform("min_child_weight",0.1,10)# minimum sum of hessian(instance weigth) needed in a child
    # "colsample_by*":hp.uniform("colsample_by*",0.5,1)# subsample ratio of features
}

In [82]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)

In [104]:
# Defining the objective function
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

def objective_xgb(args):
    xgb = XGBRegressor(**args,random_state=42)

    xgb.fit(X_train, y_train)
    score_x = cross_val_score(xgb, X_train, y_train, cv= kf, scoring="neg_mean_squared_error")

    y_pred = xgb.predict(X_test)

    print("Hyperparameters : {}".format(args)) ## This can be commented if not needed.
    # print("\n\n********Train metrics***********\n")
    # print(f"RMSE for training {rmse(score_x.mean())}\n") 
    # print(f"MSE for training {(-score_x).mean()}\n")     
    print("\n\n*********Test metrics*********\n")
    print("\t RMSE : {:.3f} \t MSE : {:.3f} \t MAE : {:.3f} \t R2 : {:.3f}".format(mean_squared_error(y_test, y_pred,squared=False),
                        mean_squared_error(y_test, y_pred),mean_absolute_error(y_test, y_pred),r2_score(y_test, y_pred)))
    # print("\t MSE : {}".format(mean_squared_error(y_test, y_pred)))
    # print("\t MAE : {}".format(mean_absolute_error(y_test, y_pred)))
    # print("\t R2 : {}".format(r2_score(y_test, y_pred)))

    return mean_squared_error(y_test, y_pred)

In [105]:
import hyperopt
trials_xgb=hyperopt.Trials()# It'll record different values of hyperparameters tried,
#objective function values during each trial, time of trials, state of the trial (success/failure), etc

results_xgb = hyperopt.fmin(objective_xgb,space=search_space_xgb,
                            algo=hyperopt.tpe.suggest,trials=trials_xgb,max_evals = 10)

#trials_xgb
# best_xgb_trial = trials_xgb.best_trial

Hyperparameters : {'alpha': 0.31656459445894236, 'eta': 0.11116580978452943, 'gamma': 0.5449757686653286, 'lambda': 0.17679696453717375, 'max_bin': 70, 'max_depth': 10, 'min_child_weight': 6.999640456328947, 'subsample': 0.5985406594177096}
                                                      

*********Test metrics*********

	 RMSE : 0.301 	 MSE : 0.091 	 MAE : 0.231 	 R2 : 0.905
Hyperparameters : {'alpha': 0.4619341207933632, 'eta': 0.021982242624377972, 'gamma': 0.92595006680113, 'lambda': 0.21146595903860455, 'max_bin': 768, 'max_depth': 9, 'min_child_weight': 5.611677353441909, 'subsample': 0.7694719470644968}
                                                                                 

*********Test metrics*********

	 RMSE : 0.389 	 MSE : 0.152 	 MAE : 0.320 	 R2 : 0.841                          
Hyperparameters : {'alpha': 0.19916060593819113, 'eta': 0.15740277091434332, 'gamma': 0.8059142419706186, 'lambda': 0.8563835498559892, 'max_bin': 192, 'max_depth': 18, 'min_child

In [107]:
# trials_xgb
# best_xgb_trial = trials_xgb.best_trial
results_xgb # best params to obtain the minimum loss which is the mean loss of all the trials

{'alpha': 0.10930597100566056,
 'eta': 0.27437013641357,
 'gamma': 0.1848843054153474,
 'lambda': 0.4824370615383845,
 'max_bin': 1890,
 'max_depth': 9,
 'min_child_weight': 8.206361979415263,
 'subsample': 0.5101813658839256}

In [110]:
# trials_xgb.trials[5]['result']
Results_xgb= trials_xgb.results
Results_xgb[:5]# first five losses

[{'loss': 0.09057758698685531, 'status': 'ok'},
 {'loss': 0.15156551562358625, 'status': 'ok'},
 {'loss': 0.09464518283231013, 'status': 'ok'},
 {'loss': 0.09052802942657448, 'status': 'ok'},
 {'loss': 0.08979729447062022, 'status': 'ok'}]

In [111]:
trials_xgb.average_best_error()# average of all differents losses of differents trials

0.07236142036153405

#### LightGBM

In [ ]:
search_space_lgbm={
    "alpha":hp.uniform("alpha",0.05,1), #L1 regularization term on weigths
    "lambda":hp.uniform("lambda",0.05,1), #L2 regularization term on weigths 
    "eta":hp.uniform("eta",0.01,0.3),# learning rate
    "max_depth":hp.uniform("max_depth",2,30),
    "subsample":hp.uniform("subsample",0.5,1),
    "max_bin":hp.uniform("max_bin",10,2000),# maximum number of descrete bins  to bucket continuous features
    "gamma":hp.uniform("gamma",0,1), # min spleat loss
    "min_child_weight":hp.uniform("min_child_weight",0.1,10),# minimum sum of hessian(instance weigth) needed in a child
    "colsample_by*":hp.uniform("colsample_by*",0.5,1)# subsample ratio of features
}

In [ ]:
# Defining the objective function
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

def objective_lgbm(args):
    lgb = LGBMRegressor(**args, random_state=42)

    lgb.fit(X_train, y_train)

    y_pred = lgb.predict(X_test)

    print("Hyperparameters : {}".format(args)) ## This can be commented if not needed.
    print("RMSE : {}\n".format(mean_squared_error(y_test, y_pred,squared=False)))
    print("MSE : {}\n".format(mean_squared_error(y_test, y_pred)))
    print("MAE : {}\n".format(mean_absolute_error(y_test, y_pred)))
    print("R2 : {}\n".format(r2_score(y_test, y_pred)))

    return 0

#### CatBoost

In [22]:
import catboost

search_space_catb={
    "alpha":hp.uniform("alpha",0.05,1), #L1 regularization term on weigths
    "lambda":hp.uniform("lambda",0.05,1), #L2 regularization term on weigths 
    "eta":hp.uniform("eta",0.01,0.3),# learning rate
    "max_depth":hp.uniform("max_depth",2,30),
    "subsample":hp.uniform("subsample",0.5,1),
    "max_bin":hp.uniform("max_bin",10,2000),# maximum number of descrete bins  to bucket continuous features
    "gamma":hp.uniform("gamma",0,1), # min spleat loss
    "min_child_weight":hp.uniform("min_child_weight",0.1,10),# minimum sum of hessian(instance weigth) needed in a child
    "colsample_by*":hp.uniform("colsample_by*",0.5,1)# subsample ratio of features
}

NameError: name 'hp' is not defined

In [ ]:
def objective_catb(args):
    catb = CatRegressor(**args, random_state=42)

    lgb.fit(X_train, y_train)

    y_pred = lgb.predict(X_test)

    print("Hyperparameters : {}".format(args)) ## This can be commented if not needed.
    print("RMSE : {}\n".format(mean_squared_error(y_test, y_pred,squared=False)))
    print("MSE : {}\n".format(mean_squared_error(y_test, y_pred)))
    print("MAE : {}\n".format(mean_absolute_error(y_test, y_pred)))
    print("R2 : {}\n".format(r2_score(y_test, y_pred)))

    return 0